Absolute magnitude is defined to be the apparent magnitude an object would have if it were located at a distance of 10 parsecs.
In astronomy, absolute magnitude (M) is a measure of the luminosity of a celestial object on an inverse logarithmic astronomical magnitude scale.

In [10]:
import time
import numpy as np
import h5py
import hdf5plugin
import pandas as pd
import matplotlib as mpl
import matplotlib.colors as cm
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from unyt import yr, Myr, kpc, arcsec, nJy, Mpc, Msun, erg, s, Hz
from astropy.cosmology import Planck18 as cosmo
from scipy import signal
import os 
import csv
import resource
import pickle
import shutil
import json
from synthesizer.grid import Grid
from synthesizer.parametric import SFH, ZDist
from synthesizer.particle.stars import sample_sfhz
from synthesizer.parametric import Stars as ParametricStars
from synthesizer.particle.particles import CoordinateGenerator
from synthesizer.filters import Filter, FilterCollection
from synthesizer.sed import combine_list_of_seds

from synthesizer.load_data.load_camels import load_CAMELS_IllustrisTNG
from synthesizer.kernel_functions import Kernel

from synthesizer.conversions import lnu_to_absolute_mag

current_directory = os.getcwd()
print("Current directory:", current_directory)

# Alternative method for LF
def calc_lf(ab_mag, volume, massBinLimits):
    try:
        hist, dummy = np.histogram(ab_mag, bins=massBinLimits)
        hist = np.float64(hist)
        phi = (hist / volume) / (massBinLimits[1] - massBinLimits[0])
        phi_sigma = (np.sqrt(hist) / volume) / (massBinLimits[1] - massBinLimits[0]) # Poisson errors
        return phi, phi_sigma, hist
    except Exception as e:
        print(f"Error in calc_lf: {e}")
        return None, None, None

# Define directories to process
directories = [f'LH_{i}' for i in range(104, 105)]
print(directories)

grid_name = "bc03-2016-Miles_chabrier-0.1,100.hdf5"
grid_dir = "/home/jovyan/"
grid = Grid(grid_name, grid_dir=grid_dir, read_lines=False)


# Define the directory where the text files will be saved
output_dir = "/home/jovyan/camels/play/synth-play/LH/output/"

# List to keep track of skipped directories
skipped_directories = []

LH_X = 'LH_104'
# Get grid for all galaxies
dir_ = '/home/jovyan/Data/Sims/IllustrisTNG/LH/' + LH_X

gals_074 = load_CAMELS_IllustrisTNG(
    dir_,
    snap_name='snapshot_074.hdf5', 
    fof_name='groups_074.hdf5',
    verbose=False,
)

# Check if stars and gas data is valid
if any(gal.stars is None or gal.gas is None for gal in gals_074):
    print(f"Skipping {LH_X} due to missing stars or gas data")
    skipped_directories.append(LH_X)
    #continue

cat_074 = dir_+'/groups_074.hdf5'
# Open file
f_h5py = h5py.File(cat_074, 'r')

# Read different attributes of the header
boxSize_074 = f_h5py['Header'].attrs[u'BoxSize']/1e3 # Mpc/h
redshift_074 = f_h5py['Header'].attrs[u'Redshift']

# Filter galaxies to only include those with 100 or more star particles
gals_074 = [gal for gal in gals_074 if gal.stars.nstars >= 100]

# Initialize lists to accumulate results
combined_lnu = None

filt1 = Filter("top_hat/filter.1", lam_min=1400, lam_max=1600, new_lam=grid.lam)
filt_lst = [filt1]
# try with one
spec = gals_074[0].stars.get_spectra_incident(grid) #this line is causing error.
abs_mag_arr = []
for gal in gals_074:
    # Get spectra incident for each galaxy
    spec = gal.stars.get_spectra_incident(grid) #this line is causing error.
    spec.get_fnu0()

    if combined_lnu is None:
        combined_lnu = spec.lnu
    else:
        combined_lnu += spec.lnu

    spec.get_photo_luminosities(filt_lst)
    abs_mag = lnu_to_absolute_mag(spec.photo_luminosities.photo_luminosities)
    abs_mag_th = abs_mag[0]
    abs_mag_arr.append(abs_mag_th)

print(abs_mag_arr)



# Calculate luminosity function
little_h =  0.6711
Vphys = (boxSize_074 / little_h) ** 3
Vcom = Vphys * ((1 + redshift_074) ** 3)

massBinLimits = np.arange(-22, -16, 0.5)
phi, phi_sigma, hist = calc_lf(abs_mag_arr, Vcom, massBinLimits)

if phi is None or phi_sigma is None or hist is None:
    raise ValueError("Error in luminosity function calculation")

massBinLimits = massBinLimits[:-1]

# Define output file path
output_file = f"{output_dir}{LH_X}.txt"

# Write the data to the text file
with open(output_file, 'w') as txtfile:
    # Write phi values
    txtfile.write("phi\n")
    for value in phi:
        txtfile.write(f"{value}\n")

    # Write phi_sigma values
    txtfile.write("phi_sigma\n")
    for value in phi_sigma:
        txtfile.write(f"{value}\n")

    # Write hist values
    txtfile.write("hist\n")
    for value in hist:
        txtfile.write(f"{value}\n")

    # Write massBinLimits values
    txtfile.write("massBinLimits\n")
    for value in massBinLimits:
        txtfile.write(f"{value}\n")

print(f'Written out: {output_file}')

# Print skipped directories at the end
if skipped_directories:
    print("Skipped directories:", skipped_directories)
